
# 1) BDV & trends - Collect and prepare data
**Goal:** This notebook creates a clean dataset from existing sources.



## 1) clean AnAge
Clean the AnAge dataset in order to test the link between max longevity residual and EBL count.


In [1]:
%matplotlib inline

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import spearmanr


ANAGE_CSV = "anage.csv"  

OUTPUT_DIR = "outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)


In [89]:
import re
import csv

RAW = "anage.csv"                       # original file
FIXED = "anage_fixed_new.csv"           # clean file we'll generate

def fix_line(line: str) -> str:
    # 1) trim whitespace/newlines
    s = line.rstrip("\r\n")

    # 2) remove leading outer quote if present
    if s.startswith('"'):
        s = s[1:]

    # 3) remove trailing sequence of quote + semicolons (e.g., '";;;;')
    s = re.sub(r'";*$', '', s)

    # 4) inside the remaining text, turn any '";"' (semicolon between quoted tokens) into a space inside the quotes
    #    e.g.  ..."Fruit";"fly"...  -> ..."Fruit fly"...
    s = s.replace('";"', '" "')

    # 5) collapse doubled quotes ("") to single "
    s = s.replace('""', '"')

    # Done. At this point, commas are true delimiters; fields remain quoted as normal CSV.
    return s

# Pass 1: fix each line and write a new CSV
with open(RAW, "r", encoding="utf-8", errors="replace") as fin, \
     open(FIXED, "w", encoding="utf-8", newline="") as fout:
    for line in fin:
        fixed = fix_line(line)
        # skip empty lines
        if fixed.strip():
            fout.write(fixed + "\n")

print(f"✅ Wrote cleaned file: {FIXED}")

# Pass 2: load the cleaned CSV with pandas
df = pd.read_csv(FIXED)

# --- Build 'Scientific name' and coerce numeric columns ---

def find_col(df, candidates):
    norm = {c: re.sub(r"[^a-z0-9]+","", c.lower()) for c in df.columns}
    for cand in candidates:
        key = re.sub(r"[^a-z0-9]+","", cand.lower())
        for c,n in norm.items():
            if n == key:
                return c
    for cand in candidates:
        key = re.sub(r"[^a-z0-9]+","", cand.lower())
        for c,n in norm.items():
            if key in n:
                return c
    raise KeyError(f"None of {candidates} found. Columns: {list(df.columns)}")

def to_num(series):
    return pd.to_numeric(
        (series.astype(str)
               .str.replace("\u00A0","", regex=False)  # NBSP
               .str.replace(" ", "", regex=False)      # thousands spaces
               .str.replace(",", ".", regex=False)),   # decimal comma -> dot
        errors="coerce"
    )


class_col   = find_col(df, ["Class"])
order_col   = find_col(df, ["Order"])
genus_col   = find_col(df, ["Genus"])
species_col = find_col(df, ["Species"])
life_col    = find_col(df, ["Maximum longevity (yrs)","Maximum.longevity.yrs"])
mass_col    = find_col(df, ["Adult weight (g)","Body.mass.g","Adult.body.mass.g"])

df["Scientific name"] = (
    df[genus_col].astype(str).str.strip() + " " +
    df[species_col].astype(str).str.strip()
)

df["Maximum longevity (yrs)"] = to_num(df[life_col])
df["Adult weight (g)"]        = to_num(df[mass_col])

anage = df[["Scientific name", order_col, "Adult weight (g)", "Maximum longevity (yrs)"]].rename(
    columns={order_col: "Order"}
)

print("Columns:", list(anage.columns))
display(anage.head())

✅ Wrote cleaned file: anage_fixed.csv
Columns: ['Scientific name', 'Order', 'Adult weight (g)', 'Maximum longevity (yrs)']


,Scientific name,Order,Adult weight (g),Maximum longevity (yrs)
0,Daphnia pulicaria,Diplostraca,NaN,0.19
1,Drosophila melanogaster,Diptera,NaN,0.30
2,Apis mellifera,Hymenoptera,NaN,8.00
3,Cardiocondyla obscurior,Hymenoptera,NaN,0.50
4,Lasius niger,Hymenoptera,NaN,28.00


 ## 2) clean EBL count
We'll take data from Kawasaki et al.'s 2021 PNAS paper and output a clean "EBL-count_per_species" CSV file.

In [98]:
PNAS_XLSX = "pnas.2026235118.sd01.xlsx"   # this file comes from the Kawasaki et al. (2021) paper

import re
import pandas as pd

# Read the sheet; this file has a 2-row explanatory header, real header is on row index 1
raw = pd.read_excel(PNAS_XLSX, sheet_name="Sheet1", header=None)

# Find the header row by locating the cell that says "Host species"
hdr_row = raw.index[raw.apply(lambda r: r.astype(str).str.contains(r"\bHost species\b", na=False)).any(axis=1)][0]
header = raw.iloc[hdr_row].tolist()
df = raw.iloc[hdr_row+1:].copy()
df.columns = header

# Keep minimal columns: EBL Name (contains the genus) and Host species
df = df.rename(columns={"Name":"EBL_Name", "Host species":"Host_species"})
df = df[["EBL_Name", "Host_species"]].dropna(subset=["EBL_Name","Host_species"])

# Extract the bornavirus genus encoded in EBL_Name strings like:
#   EBLN-Orthobornavirus.1-AotNan, EBLX-Carbovirus.3-Myotis_..., EBLP-Cultervirus...
def extract_genus(s):
    m = re.search(r"EBL\w*-([A-Za-z]+)", str(s))
    return m.group(1) if m else None

df["Virus_Genus"] = df["EBL_Name"].apply(extract_genus)
df = df.dropna(subset=["Virus_Genus"])
# Standardize capitalization (Orthobornavirus/Carbovirus/Cultervirus)
df["Virus_Genus"] = df["Virus_Genus"].str.strip().str.capitalize()

# Inspect
df.head()


# Count loci per host species x genus
wide = (
    df.groupby(["Host_species","Virus_Genus"])
      .size()
      .unstack(fill_value=0)
      .reset_index()
)

# Make sure all three columns exist even if absent in this subset
for g in ["Orthobornavirus","Carbovirus","Cultervirus"]:
    if g not in wide.columns:
        wide[g] = 0

# Add total
wide["Total_EBLs"] = wide[["Orthobornavirus","Carbovirus","Cultervirus"]].sum(axis=1)

# Keep only species with at least one EBL
wide = wide[wide["Total_EBLs"] > 0].copy()

# Save to CSV (so you can reuse without re-parsing)
wide_path = "EBL_counts_per_species.csv"
wide.to_csv(wide_path, index=False)
print("Wrote:", wide_path)
wide.head()

import re

def normalize_name(s):
    return re.sub(r"[^a-z ]", "", str(s).lower().replace("_", " ")).strip()

wide["merge_key"] = wide["Host_species"].apply(normalize_name)
mammals["merge_key"] = mammals["Scientific name"].apply(normalize_name)

Wrote: EBL_counts_per_species.csv
